In [1]:
import sources.logins as lg
from time import sleep
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
warnings.filterwarnings('ignore')

def login_exfm(driver_on = True):
    # generate key using fernet
    # key = Fernet.generate_key().decode()
    key = 'cLAiAdZU1U0sxWWK8sxhF0IWIBP4KrR3xhdu3x1EDQI='
    
    
    # login ExFM and download latest incompleted data
    name,pw = lg.save_id('exfm.txt',key)
    driver = lg.login_url('exfm',name,pw)
    
    
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search	
            break
        except Exception as e:
            print ('Wrong Username or Password. Try again...')
            name,pw = lg.save_id('exfm.txt',key)
            driver = lg.login_url('exfm',name,pw)
    
    d_type_menu = [
                    'Incompleted',
                    'History',
                    'Equipments',
                    'Customers',
                    
                    ]
    #border table
    print(f'\n{"_"*50}')
    print(f'{"|  No.|  Function": <49}|')
    print(f'|{"_"*48}|')
    for i in range(1,1+len(d_type_menu)):
        print(f'|{i: >3}  |  {d_type_menu[i-1]: <40}|')
    print(f'|{"_"*48}|') #bottom border
    while True:
        ind = str(input('Select Dataset to download: '))
        try:
            ind = int(ind)
        except:
            if ind.upper() == 'Q' or ind.upper() == 'QUIT':
                break
            else:
                print('Only accept number')
                continue
        if 0<ind<=len(d_type_menu):
            d_type = d_type_menu[ind-1]
            break
        else:
            print(f'Input number must be less than {len(d_type_menu)}\n')
    
    print(d_type)
    
    # driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search
    
    if d_type == d_type_menu[0]: # Incompleted
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[1]: # History
        Select(driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_STATUS"]')).select_by_index(0)
        print('Select Status "All" and download.')
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[2]: # Equipments
        driver.find_element_by_xpath('//*[@id="EQP_MGT_LINK"]').click() # Equipments
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    if d_type == d_type_menu[3]: # Customers
        driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click() 
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    

    # check latest file
    i_wait = 0
    while True:
        try: # check file already
            file, ctime = lg.file_latest(folder_name='Downloads')
            if file.endswith('xls'):
                print (f'\nDonwload file {file} succesful at {ctime}')
                break
            else:
                print(f'Please wait for compressing file {file}...{i_wait}s')
                i_wait +=3
                sleep(3)
        except Exception as e: #waiting for download
            print(f'Please wait for dowloading...{i_wait}s')
            i_wait +=3
            sleep(3)
    if driver_on:
        return driver
    else:
        #finish close driver
        driver.close()
    sleep(2)

def auto_accept(driver):
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    driver.switch_to.alert.accept()
    

def create_eqp(model,sn,date_installed,wty_end=''):
    #create EQP
    driver.find_element_by_xpath('//*[@id="sidADD_BUTTON_IMAGE_2"]').click()
    #search sn

    driver.find_element_by_xpath('//*[@id="sidBODY_NO"]').send_keys(sn)
    driver.find_element_by_xpath('//*[@id="sidBODY_NO"]').send_keys(Keys.RETURN)
    #click model
    driver.find_element_by_xpath('//*[@id="sidSELECT_MODEL_ACTION"]').click()

    #move to form
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('frame')
    # model typing
    driver.find_element_by_id('sidIN_NAME').send_keys(model)
    # search button
    driver.find_element_by_id('sidSEARCH_BUTTON').click()

    while True:
        try:
            driver.title
            sleep(2)
            print('Please Select Model')
        except:
            break

    # move back to main
    driver.switch_to.window(driver.window_handles[0])
    #select date installed
    driver.find_element_by_xpath('//*[@id="sidDISP_INSTALL_DATE"]').send_keys(date_installed)
    #select customer
    Select(driver.find_element_by_xpath('//*[@id="sidSTATUS_NAME"]')).select_by_index(1)
    #warranty date
    driver.find_element_by_xpath('//*[@id="sidDISP_WTY_START_DATE"]').send_keys(date_installed)
    if wty_end == '':
        wty_end = dt.strptime(date_installed,'%Y-%m-%d')+relativedelta(years=1)
        wty_end = wty_end.strftime('%Y-%m-%d')
        wty_end
    
    driver.find_element_by_xpath('//*[@id="sidDISP_WTY_END_DATE"]').send_keys(wty_end)
    sleep(0.2)
    #add equipment
    driver.find_element_by_xpath('//*[@id="sidADD_BUTTON_IMAGE"]').click()
    #return
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()

In [2]:
driver = login_exfm()


Select account:
1   |   thongle
2   |   Nguyen
3   |   FFAP0104
0   |   CREATE NEW ACCOUNT
Select account by number: 1
thongle is selected to login
Removed Downloads


Create new folder: Downloads
Select Chrome Driver and press enter for authorize Certificate...

Typing your ID...
Typing your password...
Login sucessful!


__________________________________________________
|  No.|  Function                                |
|________________________________________________|
|  1  |  Incompleted                             |
|  2  |  History                                 |
|  3  |  Equipments                              |
|  4  |  Customers                               |
|________________________________________________|
Select Dataset to download: 1
Incompleted
Please wait for dowloading...0s
Please wait for dowloading...3s

Donwload file SearchResultConsolidated.xls succesful at 26-Jan-24 10:36


In [ ]:
#click customer tab
driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click()

#search customer no.
cus_no = 'FMSV00090'#customer['No.'][0]
driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').clear()
driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').send_keys(cus_no)

#click search Button
driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()

# select first row
driver.find_element_by_xpath('//*[@id="g[0].sidDISP_CODE_field"]').click()

In [ ]:
date_installed = ''

In [ ]:
date_installed = str(input(f'Date Installed ({date_installed}): ' ) or date_installed) #'2023-06-30'
wty_end = str(input(f'WTY End: ' )) #'2023-06-30'
install_list = '''
EG-530WR	LG361K760
EG-530WR	LG361K762
EG-530WR	LG361K946
EG-530WR	LG361K948
EG-530WR	LG361K951
EG-530WR	LG361K980
EC-530WL3	KC643K558
EC-530WL3	KC643K563
'''
install_list = install_list.strip().split('\n')
i=0
for install in install_list:
    i+=1
    sn = install.strip().split('	')[1]
    model = install.strip().split('	')[0]
    print(i,model,sn)
    create_eqp(model,sn,date_installed,wty_end)
print('Done')

In [ ]:
sn_list = '''
'''
sn_list = sn_list.strip().split('\n')

In [ ]:
# Change ownership type

In [ ]:
for sn in sn_list:
    # select equipment tab
    driver.find_element_by_xpath('//*[@id="EQP_MGT_LINK"]').click()
    # type sn
    driver.find_element_by_xpath('//*[@id="sidIN_BODY_NO"]').send_keys(sn)
    #search
    driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
    #select first row
    driver.find_element_by_xpath('//*[@id="g[0].sidDISP_BODY_NO_data"]').click()
    #change ownership special
    Select(driver.find_element_by_xpath('//*[@id="sidSTATUS_NAME"]')).select_by_index(6)
    # save
    driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()

###### Read from EXP file

In [10]:
import xlwings as xw
import pathlib
import os

In [8]:
path = pathlib.Path().absolute()

In [11]:
file_name = os.path.join(path,'inspection','EQP_TBL_EXPORT_SOUTH.xlsx')

In [44]:
wb = xw.Book(file_name)

In [45]:
sh = wb.sheets('Address')

In [23]:
for i in range(2,1000):
    if sh.range(f'A{i}').value != None:
        print(sh.range(f'A{i}').value)
    else:
        break

FMSV00446
FMSV00095
FMSV00110
FMSV00276
ANP00007
FMSV00041
FMSV00209
FMSV00106
FMSV00293
FMSV00036
FMSV00388
FMSV00086
FMSV00132
FMSV00134
FMSV00133
FMSV00139
FMSV00144
FMSV00053
FMSV00087
ANP00035
FMSV00294
FMSV00284
ANP00015
FMSV00010
FMSV00336
FMSV00333
FMSV00011
FMSV00345
FMSV00401
FMSV00035
FMSV00332
FMSV00287
FMSV00338
FMSV00344
FMSV00052
FMSV00026
FMSV00298
FMSV00330
FMSV00002
FMSV00352
ANP00036
FMSV00299
FMSV00273
ANP00034
FMSV00042
FMSV00001
FMSV00308
FMSV00318
FMSV00319
ANP00019
FMSV00423
FMSV00070
FMSV00302
FMSV00148
FMSV00356
FMSV00204
FMSV00161
FMSV00221
FMSV00205
FMSV00023
FMSV00264
FMSV00410
FMSV00343
FMSV00003
FMSV00432
FMSV00122
FMSV00263
FMSV00262
FMSV00015
FMSV00051
FMSV00126
FMSV00340
FMSV00419
FMSV00171
FMSV00154
FMSV00155
FMSV00157
FMSV00404
FMSV00433
FMSV00158
FMSV00156
FMSV00249
FMSV00380
FMSV00407
FMSV00278
FMSV00246
FMSV00206
FMSV00326
FMSV00311
FMSV00342
FMSV00341
FMSV00271
FMSV00312
FMSV00289
FMSV00303
FMSV00411
FMSV00383
FMSV00365
FMSV00414
FMSV00082
FMSV00

In [22]:
sh.range('A116').value == None

True

In [26]:
#click customer tab
driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click()



In [63]:
#search customer no.
# cus_no = 'FMSV00446'#customer['No.'][0]
for i in range(k,1000):
    if sh.range(f'A{i}').value != None:
        print(i,sh.range(f'A{i}').value)
        cus_no = sh.range(f'A{i}').value
        try:
            driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').clear()
        except Exception as e:
            print(e)
            print(f'Do manual from {i}')
            k=i
            break
        driver.find_element_by_xpath('//*[@id="sidIN_ACC_CODE"]').send_keys(cus_no)

        #click search Button
        driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()

        # select first row
        driver.find_element_by_xpath('//*[@id="g[0].sidDISP_CODE_field"]').click()
        
        if sh.range(f'C{i}').value != None:
            hs_name = driver.find_element_by_xpath('//*[@id="sidNAME"]')
            hs_name.clear()
            hs_name.send_keys(sh.range(f'C{i}').value)

        #address
        addr1 = driver.find_element_by_xpath('//*[@id="sidADDRESS1"]')
        addr1.clear()
        try:
            addr1.send_keys(sh.range(f'F{i}').value)
        except:
            pass

        # city
        city = driver.find_element_by_xpath('//*[@id="sidCITY"]')
        city.clear()
        try:
            city.send_keys(sh.range(f'H{i}').value)
        except:
            pass

        # territory
        territory = driver.find_element_by_xpath('//*[@id="sidTEXT_TERRITORY_NAME"]')
        territory.clear()
        try:
            territory.send_keys(sh.range(f'E{i}').value)
        except:
            pass

        #area
        area = driver.find_element_by_xpath('//*[@id="sidTEXT_AREA_NAME"]')
        area.clear()
        try:
            area.send_keys(sh.range(f'I{i}').value)
        except:
            pass
        
        driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
        sleep(0.3)
        driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
    else:
        break


71 FMSV00051
72 FMSV00126
73 FMSV00340
74 FMSV00419
75 FMSV00171
76 FMSV00154
77 FMSV00155
78 FMSV00157
79 FMSV00404
80 FMSV00433
81 FMSV00158
82 FMSV00156
83 FMSV00249
84 FMSV00380
85 FMSV00407
86 FMSV00278
87 FMSV00246
88 FMSV00206
89 FMSV00326
90 FMSV00311
91 FMSV00342
92 FMSV00341
93 FMSV00271
94 FMSV00312
95 FMSV00289
96 FMSV00303
97 FMSV00411
98 FMSV00383
99 FMSV00365
100 FMSV00414
101 FMSV00082
102 FMSV00364
103 FMSV00409
104 FMSV00378
105 FMSV00415
106 FMSV00363
107 FMSV00384
108 FMSV00392
109 FMSV00379
110 FMSV00153
111 FMSV00385
112 FMSV00381
113 FMSV00408
114 FMSV00431
115 FMSV00175


In [57]:
k = 36

In [ ]:
# driver.find_element_by_xpath('')

In [ ]:
# driver.find_element_by_xpath('')

In [ ]:
# driver.find_element_by_xpath('')

In [ ]:
# driver.find_element_by_xpath('')